In [ ]:
pip install transformers datasets evaluate accelerate peft bitsandbytes trl -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.3/297.3 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.0/225.0 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.1/80.1 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [ ]:
from google.colab import drive
import datetime
from datasets import (load_dataset,
                      Dataset,
                      DatasetDict)
import pandas as pd
import torch
from transformers import (BitsAndBytesConfig,
                          AutoModelForCausalLM,
                          TrainingArguments,
                          Trainer,
                          AutoTokenizer,
                          set_seed)
from tqdm.auto import tqdm
from peft import (prepare_model_for_kbit_training,
                  get_peft_model,
                  LoraConfig,
                  TaskType)

import accelerate
from trl import (SFTTrainer,
                 DataCollatorForCompletionOnlyLM)

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Set all the paths and parameters which can be changed for training
base_model = "microsoft/phi-2"

set_seed(42)
timestamp = str(int(datetime.datetime.now().replace(microsecond=0).timestamp()))
print(timestamp)

filestamp = "1711835043" # Select the timestamp of the latest version
source = "news"


train_path = f'/content/drive/MyDrive/Thesis data/relevant_selection/{filestamp}_{source}_train.json'
val_path = f'/content/drive/MyDrive/Thesis data/relevant_selection/{filestamp}_{source}_val.json'

log_path = f'/content/drive/MyDrive/Thesis_models/Logs/{timestamp}_{filestamp}_{source}'

adapter_name = f"{timestamp}_{filestamp}_{source}_trained"
output_dir = f'/content/drive/MyDrive/Thesis_models/{adapter_name}'

batch_size = 2
logging_steps = 800
epochs = 10


1712334679


In [ ]:
# Data loading
train = pd.read_json(train_path)
val = pd.read_json(val_path)

train_ds = Dataset.from_pandas(train).rename_column("result_1", "labels")
val_ds = Dataset.from_pandas(val).rename_column("result_1", "labels")

data = DatasetDict()
data['train'] = train_ds
data['val'] = val_ds

del train, val, train_ds, val_ds

In [ ]:
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model,trust_remote_code=True,)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# Quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)

In [ ]:
# Load the base model
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map='auto',
)

config.json:   0%|          | 0.00/863 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
model = prepare_model_for_kbit_training(model)

In [ ]:
# Parameter efficient fine-tuning config
peft_config = LoraConfig(
                            r=32,
                            lora_alpha=32,
                            target_modules=[
                            'q_proj','k_proj','v_proj','dense',
                            ],
                            bias="none",
                            lora_dropout=0.05,
                        )


In [ ]:
# Function to format the prompts
def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['prompt'])):
        text = f"{example['prompt'][i]} {example['labels'][i]}"
        output_texts.append(text)
    return output_texts

response_template = " \n\nOutput:"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

# Training arguments config
training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=5e-6,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epochs,
    weight_decay=0.0,
    # max_grad_norm=0.5,
    evaluation_strategy="steps",
    label_names = ["labels"],
    eval_steps=logging_steps,
    logging_steps=logging_steps,
    save_steps=logging_steps,
    load_best_model_at_end=True,
)


# Trainer config
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=data["train"],
    peft_config = peft_config,
    eval_dataset=data["val"],
    formatting_func=formatting_prompts_func,
    tokenizer=tokenizer,
    data_collator=collator,
    max_seq_length=2048,

)



Map:   0%|          | 0/3600 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss


Step,Training Loss,Validation Loss
800,1.129300,0.790530
1600,0.775700,0.743992
2400,0.761800,0.741083
3200,0.752900,0.741140
4000,0.748100,0.733510
4800,0.746500,0.756705
5600,0.732200,0.733772
6400,0.736800,0.717211
7200,0.725400,0.755066
8000,0.729100,0.733636


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.

TrainOutput(global_step=18000, training_loss=0.7448697526719835, metrics={'train_runtime': 55535.054, 'train_samples_per_second': 0.648, 'train_steps_per_second': 0.324, 'total_flos': 5.116651394210611e+17, 'train_loss': 0.7448697526719835, 'epoch': 10.0})

In [ ]:
log_history = pd.DataFrame(trainer.state.log_history)
log_history.to_csv(log_path)

In [ ]:
from google.colab import runtime
runtime.unassign()